# CAMEL Role-Playing Autonomous Cooperative Agents

This is a langchain implementation of paper: "CAMEL: Communicative Agents for “Mind” Exploration of Large Scale Language Model Society".

Overview:

The rapid advancement of conversational and chat-based language models has led to remarkable progress in complex task-solving. However, their success heavily relies on human input to guide the conversation, which can be challenging and time-consuming. This paper explores the potential of building scalable techniques to facilitate autonomous cooperation among communicative agents and provide insight into their "cognitive" processes. To address the challenges of achieving autonomous cooperation, we propose a novel communicative agent framework named role-playing. Our approach involves using inception prompting to guide chat agents toward task completion while maintaining consistency with human intentions. We showcase how role-playing can be used to generate conversational data for studying the behaviors and capabilities of chat agents, providing a valuable resource for investigating conversational language models. Our contributions include introducing a novel communicative agent framework, offering a scalable approach for studying the cooperative behaviors and capabilities of multi-agent systems, and open-sourcing our library to support research on communicative agents and beyond.

The original implementation: https://github.com/lightaime/camel

Project website: https://www.camel-ai.org/

Arxiv paper: https://arxiv.org/abs/2303.17760


## Import LangChain related modules 

In [1]:
from typing import List
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    BaseMessage,
)

## Define base classes and CAMEL classes

In [20]:
class Entity:
    def reset(self):
        raise NotImplementedError
        
    def step(self):
        raise NotImplementedError

class CAMELAgent(Entity):

    def __init__(
        self,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.system_message = system_message
        self.model = model
        self.message_history = [self.system_message]

    def reset(self, message=None) -> None:
        if message is not None:
            self.message_history.append(message)
        return self.message_history

    def _update_messages(self, mbessage: BaseMessage) -> List[BaseMessage]:
        self.message_history.append(message)
        return self.message_history

    def step(
        self,
        input_message: HumanMessage,
    ) -> AIMessage:
        
        messages = self._update_messages(input_message)
        output_message = self.model(messages)
        self._update_messages(output_message)

        return output_message


In [21]:
class BaseSimulator:
    
    def reset(self):
        raise NotImplementedError 
    
    def step(self):
        raise NotImplementedError 

class CAMELSimulator(BaseSimulator):
    def __init__(self, assistant_agent, user_agent):
        self.assistant_agent = assistant_agent
        self.user_agent = user_agent
        
    def reset(self):        
        # Initialize chats
        initial_msg_from_assistant = (
            "Now start to give me instructions one by one. "
            "Only reply with Instruction and Input.")
        msg_to_user = HumanMessage(content=initial_msg_from_assistant)
        msg_from_assistant = AIMessage(content=initial_msg_from_assistant)
        
        # Reset agents
        assistant_agent.reset(msg_from_assistant)
        user_agent.reset()
        
        return msg_to_user, msg_from_assistant
    
    def step(self, msg_to_user, msg_to_assistant):

        msg_from_user = user_agent.step(msg_to_user)    
        msg_to_assistant = HumanMessage(content=msg_from_user.content)

        msg_from_assistant = assistant_agent.step(msg_to_assistant)
        msg_to_user = HumanMessage(content=msg_from_assistant.content)

        return msg_to_user, msg_to_assistant

## Setup roles and task for role-playing

In [14]:
assistant_role_name = "Python Programmer"
user_role_name = "Stock Trader"
task = "Develop a trading bot for the stock market"
word_limit = 50 # word limit for task brainstorming

## Create a task specify agent for brainstorming and get the specified task

In [15]:
task_specifier_prompt = [
    SystemMessage(content="You can make a task more specific."),
    HumanMessage(content=
        f"""Here is a task that {assistant_role_name} will help {user_role_name} to complete: {task}.
        Please make it more specific. Be creative and imaginative.
        Please reply with the specified task in {word_limit} words or less. Do not add anything else."""
        )
]
specified_task = ChatOpenAI(temperature=1.0)(task_specifier_prompt).content

[SystemMessage(content='You can make a task more specific.', additional_kwargs={}), HumanMessage(content='Here is a task that Python Programmer will help Stock Trader to complete: Develop a trading bot for the stock market.\n        Please make it more specific. Be creative and imaginative.\n        Please reply with the specified task in 50 words or less. Do not add anything else.', additional_kwargs={})]
Develop a Python-based trading bot for the S&P 500 stock market that uses machine learning algorithms to analyze market trends and make real-time trades based on risk management parameters and company financial data.


## Create system messages for AI assistant and AI user

In [16]:
def generate_sys_msgs(assistant_role_name: str, user_role_name: str, task: str):
    
    # the prompt that the user gives to the assistant
    assistant_inception_prompt = (
    f"""Never forget you are a {assistant_role_name} and I am a {user_role_name}. Never flip roles! Never instruct me!
    We share a common interest in collaborating to successfully complete a task.
    You must help me to complete the task.
    Here is the task: {task} 
    Never forget our task!
    I must instruct you based on your expertise and my needs to complete the task.

    I must give you one instruction at a time.
    You must write a specific solution that appropriately completes the requested instruction.
    You must decline my instruction honestly if you cannot perform the instruction due to physical, moral, legal reasons or your capability and explain the reasons.
    Do not add anything else other than your solution to my instruction.
    You are never supposed to ask me any questions you only answer questions.
    You are never supposed to reply with a flake solution. Explain your solutions.
    Your solution must be declarative sentences and simple present tense.
    Unless I say the task is completed, you should always start with:

    Solution: <YOUR_SOLUTION>

    <YOUR_SOLUTION> should be specific and provide preferable implementations and examples for task-solving.
    Always end <YOUR_SOLUTION> with: Next request."""
    )

    # the prompt that the assistant gives to the user
    user_inception_prompt = (
    f"""Never forget you are a {user_role_name} and I am a {assistant_role_name}. Never flip roles! You will always instruct me.
    We share a common interest in collaborating to successfully complete a task.
    I must help you to complete the task.
    Here is the task: {task} 
    Never forget our task!
    You must instruct me based on my expertise and your needs to complete the task ONLY in the following two ways:

    1. Instruct with a necessary input:
    Instruction: <YOUR_INSTRUCTION>
    Input: <YOUR_INPUT>

    2. Instruct without any input:
    Instruction: <YOUR_INSTRUCTION>
    Input: None

    The "Instruction" describes a task or question. The paired "Input" provides further context or information for the requested "Instruction".

    You must give me one instruction at a time.
    I must write a response that appropriately completes the requested instruction.
    I must decline your instruction honestly if I cannot perform the instruction due to physical, moral, legal reasons or my capability and explain the reasons.
    You should instruct me not ask me questions.
    Now you must start to instruct me using the two ways described above.
    Do not add anything else other than your instruction and the optional corresponding input!
    Keep giving me instructions and necessary inputs until you think the task is completed.
    When the task is completed, you must only reply with a single word <CAMEL_TASK_DONE>.
    Never say <CAMEL_TASK_DONE> unless my responses have solved your task."""
    )
    
    return SystemMessage(content=assistant_inception_prompt), SystemMessage(content=user_inception_prompt) 


## Create AI assistant agent and AI user agent

In [17]:
assistant_sys_msg, user_sys_msg = generate_sys_msgs(assistant_role_name, user_role_name, specified_task)
assistant_agent = CAMELAgent(assistant_sys_msg, ChatOpenAI(temperature=0.2))
user_agent = CAMELAgent(user_sys_msg, ChatOpenAI(temperature=0.2))

## Start role-playing session to solve the task!

In [19]:
print(f"Original task prompt:\n{task}\n")
print(f"Specified task prompt:\n{specified_task}\n")

max_iters = 2
n = 0

simulator = CAMELSimulator(assistant_agent, user_agent)
msg_to_user, msg_to_assistant = simulator.reset()

while n < max_iters:
    msg_to_user, msg_to_assistant = simulator.step(
        msg_to_user, msg_to_assistant)
    print(f"AI User ({user_role_name}):\n\n{msg_to_assistant.content}\n\n")
    print(f"AI Assistant ({assistant_role_name}):\n\n{msg_to_user.content}\n\n")

    if "<CAMEL_TASK_DONE>" in msg_to_assistant.content:
        break
        
    n += 1

Original task prompt:
Develop a trading bot for the stock market

Specified task prompt:
Develop a Python-based trading bot for the S&P 500 stock market that uses machine learning algorithms to analyze market trends and make real-time trades based on risk management parameters and company financial data.

AI User (Stock Trader):

Instruction: Install the necessary libraries for the project.
Input: None.


AI Assistant (Python Programmer):

Solution: To install the necessary libraries for the project, we can use pip, the package installer for Python. We can create a virtual environment for the project and install the required libraries using the requirements.txt file. Here are the commands to execute in the terminal:

1. Create a virtual environment: 
   python3 -m venv myenv
2. Activate the virtual environment:
   source myenv/bin/activate
3. Install the required libraries:
   pip install -r requirements.txt

Note: The requirements.txt file should contain the names of all the required 